In [12]:

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.service import Service 
import time
import pandas as pd
import random 
import csv 
from pathlib import Path

#!pip3 install 2captcha-python

import sys
import os

from twocaptcha import TwoCaptcha




In [13]:
option = webdriver.FirefoxOptions()
option.set_preference("browser.download.manager.showWhenStarting", False)
option.set_preference('browser.helperApps.neverAsk.saveToDisk',
                       'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                       ';application/vnd.ms-excel'
                       ';application/xls'
                       ';application/xlsx')


In [14]:
# Open browser and hit the link

driver= webdriver.Firefox(options=option)

driver.get('https://mppanchayatdarpan.gov.in/PanchParmeshwar/Public/GetWorksListAll.aspx?DistId=MTQ=&LBId=MjQxMDg=&GPId=NjEyMA==&TypeId=MA==')




In [15]:
#function to solve captcha 
def Captcha_Solve():
    import sys
    from twocaptcha import TwoCaptcha
    result = None
    
    sitekey = '<your_site_key_from_two_captcha>'
    api_key = '<your_api_key_from_two_captcha>'
    solver = TwoCaptcha(api_key)
    try:
        result = solver.recaptcha(
            sitekey=sitekey,
            url='https://www.deezer.com/en/login'
        )

    except Exception as e:
        sys.exit(e)

    return result

In [18]:
## select the elements of dropdown menu 

##step-1. Select District 

select_district = driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_dddistrict"]')  #get the select element   

options = select_district.find_elements(By.TAG_NAME, "option") #get all the options into a list

#save into a list to get the name of the district 
Districts = [i.text for i in options]

#check if it is printing all the districts name correctly 
#print(optionsList)

#Now select each district and click in iteration 
for i in range(0, len(Districts), 1):

    ###print("starting loop on option %s" % i)
    
    print(Districts[i])
    #select the district  
    select_D = Select(driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_dddistrict"]'))

    ##condition to stop the loop 
    if i <53:
       select_D.select_by_index(i+1)

       time.sleep(1)

       #Step-2 select the localbody/block name 

       select_Local_Body = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddlb"]')))
       
       option_LB = select_Local_Body.find_elements(By.TAG_NAME, "option")

       LocalBody = [i.text for i in option_LB]

       print(LocalBody)

       for j in range(1, len(LocalBody)): 
           
           print(LocalBody[j])

           select_LB= Select(driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddlb"]'))
              
           #select a localbody 
           select_LB.select_by_index(j)


           ##Step-3 For Gram Panchayat 
           select_Gram_Panchayat = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddlgz"]')))
       
           option_GP = select_Local_Body.find_elements(By.TAG_NAME, "option")

           #Save all GP under the block in a list 
           Gram_Panchayat = [i.text for i in option_GP]

           #print(Gram_Panchayat)

           for k in range(1, len(Gram_Panchayat)): 

               print(Gram_Panchayat[k])

               #select a Gram Panchayat and iterate over 

               select_GP = Select(driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddlgz"]'))

               select_GP.select_by_index(k)

               
               #step-4. For Village Selection 

               select_village = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddvw"]')))
            
               option_VW = select_village.find_elements(By.TAG_NAME, "option")

               Village = [i.text for i in option_VW]

               #print(Village)
               
               for l in range(1, len(Village)): 
                   
                   print(Village[l])
                   
                   #select a Village 

                   select_VW = Select(driver.find_element(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_ddvw"]'))
                    
                   select_VW.select_by_index(l)

                   #select the scheme (static)
                   scheme = Select(driver.find_element(By.ID, 'ctl00_ctl00_ContentPlaceHolder1_cph_ddlScheme'))

                   scheme.select_by_visible_text('Pradhanmantri Khanij Kshetra Kalyan Yojna (PMKKKY)')
              
                   #scheme.select_by_index(64)  #or select by index 
                   

                   ##Solve Captcha 
                    
                   image = driver.find_element(By.TAG_NAME, 'img')

                   captcha = image.screenshot('captcha.png')
               
                   

                   api_key = os.getenv('564656fee9ba039abbfdba89a937b55c')

                   solver = TwoCaptcha(api_key)

                   try:
                        result = solver.normal('captcha.png')

                   except Exception as e:
                          print(e)

                   else:
                          code = result['code']
                          print(code)

                          WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'name')))

                          driver.find_element(By.TAG_NAME, "name").send_keys(code)
                           
                          ##click the search button 

                          submit_button = driver.find_element(By.CLASS_NAME, "icon-white icon-search").click()

                   time.sleep(20)

                   ##look for output and condition it 

                   #Get all the urls available in the table 

                   project = driver.find_elements(By.XPATH, '//*[@id="ctl00_ctl00_ContentPlaceHolder1_cph_gvWorks_ctl02_btnSelect"]')

                   project_urls = [i.get_attribute("href") for i in project]
                   
                   for url in project_urls: 

                         #click on the url 

                         print(url)
                          
                         driver.get(url)

                         time.sleep(1) 

                         driver.back()
    else: 
        break 




['- Select -', 'BAROD', 'NALKHEDA', 'SUSNER', 'AGAR']
'TwoCaptcha' object has no attribute 'normal'
None


InvalidArgumentException: Message: invalid type: null, expected a string at line 1 column 12
